In [1]:
%matplotlib inline

/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Cats vs Dogs redux

Attempting to configure files etc:

> NOTE: You will need to first set up these dependencies: 
- install the kaggle cli with: `pip install kaggle-cli`
- then set kaggle username and password with `kg config -u <username> -p <password>` (see https://github.com/floydwch/kaggle-cli for details). You will also need to have gone to kaggle and accepted the terms for this challenge before you can download the data.`
- apt-get install unzip

Downloading the data

In [6]:
!kg download -c dogs-vs-cats-redux-kernels-edition

Starting new HTTPS connection (1): www.kaggle.com
downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/test.zip

Starting new HTTPS connection (1): kaggle2.blob.core.windows.net
test.zip already downloaded !
downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip

train.zip already downloaded !
downloading https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/sample_submission.csv

sample_submission.csv already downloaded !


Moving the data into the proper directory

In [14]:
%%bash
mkdir -p data/catsdogs
mv sample_submission.csv data/catsdogs
mv test.zip data/catsdogs
mv train.zip data/catsdogs
cd data/catsdogs
unzip -q ./test.zip
unzip -q ./train.zip

Formatting the file structure

In [12]:
%%bash
function mv_n_files { # amount, srcDir, targetDir
    for file in $(ls -p $2 | grep -v / | tail -$1)
    do
    mv $2/$file $3
    done
}

function cp_n_files { # amount, srcDir, targetDir
    for file in $(ls -p $2 | grep -v / | tail -$1)
    do
    cp $2/$file $3
    done
}

# Split train into cat and dog directories
cd data/catsdogs
mkdir -p train/cat
mkdir -p train/dog
mv train/cat.* train/cat
mv train/dog.* train/dog

# Create validation set
mkdir -p valid/cat
mkdir -p valid/dog
mv_n_files 1000 train/cat valid/cat
mv_n_files 1000 train/dog valid/dog

# Create sample working directory
mkdir -p sample/train/cat
mkdir -p sample/train/dog
mkdir -p sample/valid/cat
mkdir -p sample/valid/dog
mkdir -p sample/test
sample_size=10
cp_n_files $sample_size train/cat sample/train/cat
cp_n_files $sample_size train/dog sample/train/dog
cp_n_files $sample_size valid/cat sample/valid/cat
cp_n_files $sample_size valid/dog sample/valid/dog
cp_n_files $sample_size test sample/test


mv: rename train/cat.* to train/cat/cat.*: No such file or directory
mv: rename train/dog.* to train/dog/dog.*: No such file or directory


Set up sample directory

Setting up the data path:

In [4]:
dataPath = "data/dogscats/sample"

## Creating the VGG classifier

In [7]:
from __future__ import division,print_function

batch_size = 64

# Add to python path dynamically
import sys
sys.path.insert(0, '../../deeplearning1/nbs')

import vgg16; reload(vgg16)